In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [3]:
keywords = ['python', 'парсинг','я'] # искомы слова

In [5]:
# не используется
def all_articles(url='https://habr.com/ru/all/'):
    """
    эта функция возвращяет все статьи
    """
    rec = requests.get(url) # получаю всю страницу
    soup = BeautifulSoup(rec.text) # перевожу в суп обьект
    articles = soup.find_all('article',class_='post_preview') # вытаскиваю все статьи
    return articles

In [9]:
df = pd.DataFrame() # общий датафрейм
pages = int(input('введити кол-во страниц для поиска статей: ')) # выбираю количество страниц по которым будет вестись поиск

for page in range(1,pages+1): # перебираю их
    url = f'https://habr.com/ru/all/page{page}/'
    rec = requests.get(url)
    soup = BeautifulSoup(rec.text)
    all_articles = soup.find_all('article',class_='post_preview') # вытаскиваю все статьи со страницы
    
    for article in all_articles: # перебираю статьи
        text = article.find('div',class_='post__text').text # нахожу текст превью
        text = text.lower()
        for i in keywords: # перебираю искомые слова
            if i in text:
                date = article.find('span',class_='post__time').text # достаю время
                heading = article.find('h2',class_='post__title').text # достаю заголовок
                heading = heading.strip() # чищю заголовок
                link = article.find('h2',class_='post__title') # добираюсь до ссылки
                link = link.find('a').get('href') # достаю ссылку

                row = pd.DataFrame({'date': [date],'head': [heading],'link': [link]}) # делаю из этого датафрейм 
                df = pd.concat([df, row]) # добавляю его к основному

df = df.drop_duplicates() # удоляю дубликаты
df.head() 

введити кол-во страниц для поиска статей: 3


,date,head,link
0,сегодня в 17:10,Книга «Создаем динамические веб-сайты на PHP. ...,https://habr.com/ru/company/piter/blog/558684/
0,сегодня в 17:08,Новая математика чёрных дыр: ещё один шаг вперед,https://habr.com/ru/company/skillfactory/blog/...
0,сегодня в 16:56,Рекомендуем город для путешествия при помощи н...,https://habr.com/ru/post/555174/
0,сегодня в 16:20,Как команда it-animals в финале Цифрового Прор...,https://habr.com/ru/post/558962/
0,сегодня в 16:10,Asciidoc для ЕСКД,https://habr.com/ru/post/558940/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [3]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

emails = ['xxx@x.ru', 'yyy@y.com'] # выбираю адреса
params = {'emailAddresses': emails} # задаю их в параметры
params = json.dumps(params) # преобразую тх

heads = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
          } # указываю заголовки


In [4]:
rec = requests.post(url,data=params,headers=heads) # делаю запрос передавая параметры

In [5]:
rec = json.loads(rec.text) # мне выдаёт словарь в строчке и я его преобразую просто в словарь
rec

{'breaches': {'16613': {'breachId': 16613,
   'site': 'verifications.io',
   'recordsCount': 677914246,
   'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n',
   'publishDate': '2019-03-28T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}},
  '17609': {'breachId': 17609,
   'site': 'vk.com',
   'recordsCount': 42536910,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and

In [7]:
# получаю email и breachId(некий id)
emails_leak = []

for a in rec['data']:
    for b in rec['data'][a]:
        for breach_id in rec['data'][a][b]:
            emails_leak.append({
                            'email':b,
                            'breachId':breach_id['breachId'] # добираюсь до breachId
                                })
emails_leak = pd.DataFrame(emails_leak) # делаю из всего датафрейм
emails_leak.head()

,email,breachId
0,xxx@x.ru,3176
1,xxx@x.ru,16613
2,yyy@y.com,16613
3,xxx@x.ru,3
4,yyy@y.com,3


In [16]:
description = []
for key,value in rec['breaches'].items():
    description.append({
                'breachId':value['breachId'],
                'site':value['site'],
                'description':value['description'],
                'date':value['publishDate']
                })

description = pd.DataFrame(description)
description.head()

,breachId,site,description,date
0,16613,verifications.io,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
1,17609,vk.com,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z
2,16488,www.dangdang.com,"This is a list of email addresses only, and as...",2019-02-21T00:00:00Z
3,18155,123rf.com,"In March 2020, the stock image agency 123RF wa...",2020-11-19T00:00:00Z
4,17110,azcentral.com,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z


In [15]:
full_df = emails_leak.merge(description, on='breachId',how='left') # соеденяю по id 
full_df = full_df[['email','site','description','date']]
full_df.head()

,email,site,description,date
0,xxx@x.ru,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z
1,xxx@x.ru,verifications.io,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
2,yyy@y.com,verifications.io,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
3,xxx@x.ru,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z
4,yyy@y.com,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
